In [128]:
learning_rate = 0.001
num_epochs = 10
batch_size = 64

In [129]:
import torch
import torch.nn as nn
import numpy as np
import os
import pandas as pd
import dask.dataframe as dd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from IPython.display import clear_output
import time

In [130]:
class MyDataset(Dataset):
    def __init__(self, file_path, chunk_size):
        self.file_path = file_path
        self.chunk_size = chunk_size
        
        self.columns = pd.read_csv(self.file_path, nrows=0).columns.str.strip().str.replace('\xa0', ' ').tolist()
        self.train_columns = [col for col in self.columns if col != 'Discard']  # 排除 'Discard' 欄位
        self.num_samples = sum(1 for _ in open(self.file_path, encoding='utf-8')) - 1  # 計算總樣本數
        
    def __getitem__(self, idx):
        chunk_start = idx // self.chunk_size * self.chunk_size
        df = pd.read_csv(self.file_path, skiprows=chunk_start + 1, nrows=self.chunk_size, header=None, encoding='utf-8')  # 跳過標題和之前的行

        df.columns = self.columns
        
        # 確保 Reward 欄位存在
        if 'Discard' not in df.columns:
            raise KeyError(f"Chunk starting at row {chunk_start + 1} does not contain 'Discard' column.")
        
            
        sample_idx = idx % self.chunk_size
        
        train_data = df[self.train_columns].iloc[sample_idx].values  # 提取除了 'Reward' 之外的欄位
        
        value_data = df['Discard'].iloc[sample_idx]  # 提取 'Reward' 欄位
        
        return torch.tensor(train_data, dtype=torch.float32), torch.tensor(value_data, dtype=torch.float32)
        
    def __len__(self):
        return self.num_samples

In [131]:
"""
讀取csv
"""
file_path = 'E:/專題/data/2021/DiscardDataAll.csv'
dataset = MyDataset(file_path, chunk_size=50000)  # 每次只加載 50000 行
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [132]:
class MyCNN(nn.Module):
    def __init__(self, num_layers=10):
        super(MyCNN, self).__init__()
        
        self.conv_layers = nn.ModuleList()
        self.conv_layers.append(nn.Conv1d(in_channels=1, out_channels=256, kernel_size=3, padding=1))
        
        for _ in range(1, num_layers - 1):
            self.conv_layers.append(nn.Conv1d(in_channels=256, out_channels=256, kernel_size=3, padding=1))
        
        self.conv_layers.append(nn.Conv1d(in_channels=256, out_channels=1, kernel_size=1, padding=0))
    
    def forward(self, x):
        for i, conv in enumerate(self.conv_layers):
            if(i < len(self.conv_layers) - 1):
                x = F.relu(conv(x))
            else:
                x = conv(x)
        x = x.squeeze(1)
        return x

In [133]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [134]:
model = MyCNN().to(device)
loss_criterion  = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
current_loss = 0
for epoch in range(num_epochs):
    for train_data, value_data in train_dataloader:
        torch.autograd.set_detect_anomaly(True)
        train_data = train_data.unsqueeze(1)
        start_time = time.time()
        train_data = train_data.to(device)
        value_data = value_data.to(device)
        output = model(train_data)
        output = output.float()
        value_data = value_data.long()
        probabilities = F.softmax(output, dim=1)
        
        predicted_labels = torch.argmax(probabilities, dim=1)
        correct_predictions = (predicted_labels == value_data).sum().item()
        accuracy = correct_predictions / value_data.size(0)
        
        loss = loss_criterion(probabilities, value_data)
        
        current_loss = loss
        
        optimizer.zero_grad()
        loss.backward()
        
        optimizer.step()
        
        clear_output()
        end_time = time.time()
        print(predicted_labels)
        print(value_data)
        print(f"Correct Predictions: {correct_predictions}")
        print(f"Total Predictions: {value_data.size(0)}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"At epoch: {epoch}, loss: {current_loss}")
        print(f"執行時間: {end_time - start_time} 秒")

tensor([28,  5, 31,  8,  2,  9, 17, 30, 16,  9, 17, 30, 16,  5, 29, 30, 19,  9,
        17, 30, 25,  9, 32, 30, 31,  9, 17, 30, 16,  9, 29, 30, 25,  9, 17, 33,
        16,  9, 17, 31, 31, 28, 28, 27, 31, 28, 28, 27, 31, 13, 28, 28, 10, 33,
        31, 32, 32, 32, 10, 33, 31, 32,  2, 24], device='cuda:0')
tensor([28,  2, 31, 23,  2, 28, 21, 23,  0,  0, 13, 13,  7, 18, 15, 15, 15, 26,
         0, 30, 25, 10, 32, 25, 31,  7,  1, 25, 18, 15, 23,  7, 25,  5, 27, 27,
        17, 13,  3, 29, 17, 29, 18, 17, 29,  8, 18, 31, 27, 27, 28, 31, 17, 27,
         8, 26, 32, 13,  1, 33, 31, 21, 25, 19], device='cuda:0')
Correct Predictions: 12
Total Predictions: 64
Accuracy: 0.1875
At epoch: 0, loss: 3.387150764465332
執行時間: 0.1093742847442627 秒
